In [174]:
# -*- coding: utf-8 -*-
import os
import re
import pandas as pd
import graphviz

# Phonology sets
h_tone = set("áéíóú")
l_tone = set("àèìòù")
f_tone = set("âêîôû")
r_tone = set("ǎěǐǒǔ")
untoned = set("aeiou")  # For long vowels (â indicates a short vowel with fall tone; âa a long F)
vowels = h_tone | l_tone | r_tone | f_tone | untoned
moraic_coda = 1  # 1 if coda carries mora, else 0

tones = h_tone | l_tone  # Combine high and low tones
special_tones = r_tone | f_tone  # Special tones (F, R)
max_syl_weight = 3 #default


class Autorep:
    def __init__(self, word="", ocp_mel="", assoc=None):
        """
        Initialize an Autorep object.

        Parameters:
        - word (str): The word with tone markers.
        - tone (str): The tone markers directly extracted from the word (HFLR).
        - mel (str): The melody (F -> HL and R -> LH) before OCP.
        - ocp_mel (str): The OCP-applied tone representation of the word.
        - assoc (list): A list of tuples (j, k) indicating the association 
                        between tone (indexed by j), mora (indexed by i), 
                        and syllable (indexed by k).
        """
        self.word = word
        self.tone = ""
        self.mel = ""
        self.ocp_mel = ocp_mel
        
        self.assoc = self.sort_assoc(assoc if assoc is not None else [])

        self.tone_labels = {"H": h_tone, "L": l_tone, "F": f_tone, "R": r_tone}
        if self.word:
            self._process_word()

    def _process_word(self):
        """Process the word to extract tones, assign associations, and apply OCP."""
        syllables = self.word.split(".")
        self.tone = "".join(
            next((k for k, v in self.tone_labels.items() if seg in v), "") 
            for seg in self.word
        )

        mora_idx = 0
        for i, syl in enumerate(syllables):
            syl_weight = self.check_coda(syl) + self.vowel_count(syl)
            for j in range(syl_weight):
                self.assoc.append((self.tone[i], j + 1, i + 1))
            #mora_idx += syl_weight

        self._flatten_tones(syllables)  # convert F and R into HL and LH
        self.mel = "".join(tone for tone, _, _ in self.assoc)
        self.ocp_mel = re.sub(r"(.)\1+", r"\1", self.mel)
        self._update_tone_indices()

    def _flatten_tones(self, syllables):
        """Flatten F and R tones into corresponding H/L patterns."""
        for i, (t, m, s) in enumerate(self.assoc):
            if self.tone[s - 1] in {"F", "R"}:
                mora_sum = sum(
                    self.check_coda(syl) + self.vowel_count(syl)
                    for syl in syllables[: s - 1]
                )
                if m - mora_sum == 1:
                    t = "H" if self.tone[s - 1] == "F" else "L"
                else:
                    t = "L" if self.tone[s - 1] == "F" else "H"
                self.assoc[i] = (t, m, s)

    def _update_tone_indices(self):
        """Update tone indices in association list to match OCP melody."""
        j, i = 0, 0
        while j < len(self.assoc) and i < len(self.ocp_mel):
            if self.assoc[j][0] == self.ocp_mel[i]:
                t, m, s = self.assoc[j]
                self.assoc[j] = (i + 1, m, s)  # Update tone index
                j += 1
            else:
                i += 1

    def get_rightmost(self, target):
        index_map = {'t': 0, 'm': 1, 's': 2}
        if target in index_map:
            return max(
                (item[index_map[target]] for item in self.assoc if item[index_map[target]] is not None), 
                default=0
            )
        return 0  # Return 0 for invalid target


    @staticmethod
    def check_coda(syl):
        """Check if a syllable contains a coda."""
        for i in range(1, len(syl)):
            if syl[i] not in vowels and syl[i - 1] in vowels:
                return moraic_coda
        return 0

    @staticmethod
    def vowel_count(syl):
        """Count the number of vowels and adjust for special tones."""
        count = 0
        for i, char in enumerate(syl):
            if char in vowels or char in tones:
                count += 1
            elif (
                char in special_tones
                and i + 1 < len(syl)
                and syl[i + 1] not in vowels
            ):
                count += 2
        return count

    @staticmethod
    def mora_count(string):
        """Count the number of mora in a string."""
        mora_count = 0
        mora_list = []
        syllables = string.split(".")
        for syl in syllables:
            syl_weight = Autorep.check_coda(syl) + Autorep.vowel_count(syl)
            mora_list.append(syl_weight)
            mora_count += syl_weight
        return mora_count, mora_list

    @staticmethod
    def contour_count(s):
        """Count the number of contour tones in a string."""
        return sum(1 for char in s if char in special_tones)

    @staticmethod
    def index_reset(lst):   

        """Reset indices of the association list to start from 1."""
        if not lst:
            return []
        
        t_shift = min((t for t, _, _ in lst if t is not None), default=None)
        m_shift = min((m for _, m, _ in lst if m is not None), default=None)
        s_shift = min((s for _, _, s in lst if s is not None), default=None)


        return [
            (
                (t - t_shift + 1) if t else None,
                (m - m_shift + 1) if m else None,
                (s - s_shift + 1) if s else None,
            )
            for t, m, s in lst
        ]

    @staticmethod
    def sort_assoc(assoc):
        def custom_compare(x):
            return float('inf') if x is None else x

        sorted_assoc = sorted(
            assoc, 
            key=lambda x: (custom_compare(x[0]), custom_compare(x[2]), custom_compare(x[1])))
        return sorted_assoc
        
    def check_empty(self):
        """Check if the object is empty."""
        return not (self.word or self.assoc or self.mel or self.ocp_mel)

    def check_contain(self, ar):
        """
        Check if the Autorep object `self` contains another Autorep object `ar`.
        Handles `None` as a wildcard in `ar`.
        """
        if ar.check_empty():
            return True

        if self.check_empty() or len(ar.assoc) > len(self.assoc):
            return False
    

        # Check if `ar.ocp_mel` is a substring of `self.ocp_mel`
        if ar.ocp_mel in self.ocp_mel:

            # Find match positions in `self.ocp_mel`
            match_positions = [
                1 + m.start() for m in re.finditer(f"(?={ar.ocp_mel})", self.ocp_mel)
            ]
            #print(match_positions)
            # Iterate over match positions
            for i_match in match_positions:
                window_size = len(ar.assoc)

                # Extract the relevant subset of `self.assoc`
                restriction = [
                    tup for tup in self.assoc if tup[0] == i_match
                ]
                #print(restriction)

                for i, tup in enumerate(restriction):
                    start_index = self.assoc.index(tup)
                    subset = self.assoc[start_index : start_index + window_size]
                    #print(subset)
                    for a, b in zip(self.index_reset(subset), ar.assoc):
                        #print(a,b, a==b)
                        
                        match = all([
                            a[0] == b[0] or b[0] is None,
                            a[1] == b[1] or b[1] is None,
                            a[2] == b[2] or b[2] is None
                        ])
                            
                    if match:
                        return True   
                    else:
                        return False
   
        else:
            return False



    
    
    def add_tone(self):
        """
        Add an unassociated tone in the AR by updating the melody and the association list.
        
        - A new tone ('H' or 'L') is added to the melody.
        - A new association (j, None, None) is added, where:
            - j is one-unit higher than the previous tone's number or 1 if starting fresh.
            - 'None' indicates the syllable is not associated with any tone unit.
        """
        # Copy the existing associations to avoid modifying the original
        new_assoc = self.assoc.copy()
        H_tone = Autorep(ocp_mel='H', assoc=new_assoc + [(1, None, None)])
        L_tone = Autorep(ocp_mel='L', assoc=new_assoc + [(1, None, None)])
        
        # Determine the next tone to add
        if not self.ocp_mel:  # Empty string case
            print('there')
            return [H_tone,L_tone]
        
        else:
            next_tone = 'H' if self.ocp_mel[-1] == 'L' else 'L'
            next_tone_index = self.get_rightmost('t') + 1
            
            # Create the updated autorep
            return [Autorep(
                ocp_mel=self.ocp_mel + next_tone,
                assoc=new_assoc + [(next_tone_index, None, None)]
            )]
        

    def add_syl(self,weight):
        new_assoc = self.assoc.copy()
        next_syl_index = self.get_rightmost('s') + 1 if self.get_rightmost('s') is not None else 1
        for i in range(weight):
            new_assoc.append((None, i+1, next_syl_index))
        
        new_ar = Autorep(ocp_mel= self.ocp_mel,assoc = new_assoc)
        return new_ar
    
    
    def check_float(self, target):
        if target == 't':
        # Check for floating tones (t exists, but m and s are None)
            return any(t and not m and not s for (t, m, s) in self.assoc)
        elif target == 's':
        # Check for floating syllables (s exists, but t is None)
            return any(not t and m and s for (t, m, s) in self.assoc)
        else:
        # Invalid target
            raise ValueError("Target must be 't' (tone) or 's' (syllable).")

            

    def float_tone_to_syl(self):
        
        """
        Associate the first floating tone to the last syllable.

        Example:
        ('HLHL', [(1, 1, 1), (2, 1, 2), (2, 2, 2), (3, None, None), (4, None, None)]) -->
        ('HLHL', [(1, 1, 1), (2, 1, 2), (2, 2, 2), (3, 2, 2), (4, None, None)])
        """
        # Check if there are any floating tones
        floating_tones = [(t, m, s) for (t, m, s) in self.assoc if not m and not s]
        # Check if there are any fixed tones
        floating_tones = [(t, m, s) for (t, m, s) in self.assoc if not m and not s]

        if not floating_tones:
            
            return None  # No floating tones to associate

        # Find the first floating tone
        if self.fully_spec():

            min_t = min(
                (t for (t, _, _) in floating_tones),
                default=None,
            )

            # Get the last fully_specified syllable's indices
            max_m, max_s = max(
                ((m,s) for (t, m, s) in self.assoc if t and m and s),
                default=(None, None),
            )

            # Update the floating tone to associate with the last syllable
            new_assoc = self.assoc[:]
            for i, (t, m, s) in enumerate(new_assoc):
                if t == min_t:
                    new_assoc[i] = (min_t, max_m, max_s)
            return(new_assoc)

           

    def float_syl_to_tone(self):
        #print("connect float syllable to fixed tone")
        """
        Associate the first floating syllable to the last tone.

        Example:
        ('HL', [(1, 1, 1), (2, 1, 2), (2, 2, 2), (None, 1, 3), (None, 2, 3)]) -->
        ('HLH', [(1, 1, 1), (2, 1, 2), (2, 2, 2), (2, 1, 3), (2, 2, 3)])
        """
        # Check if there are any floating syllables
        floating_syllables = [
            (t, m, s) for (t, m, s) in self.assoc if not t and s
        ]

        if not floating_syllables:
            return None  # No floating syllables to associate

        if any(
            t is not None 
            and m is not None 
            and s is not None for t, m, s in self.assoc
            ):

            # Find the last tone in the association list
            max_t = max(
                (t for (t, _, s) in self.assoc if t and s),
                default=None,
            )

            #Find the first floating syllable in the list

            min_s = min(
                (s for (t, _, s) in floating_syllables),
                default=None,
            )
            
            min_m = min(
                (m for (_, m, _) in floating_syllables ),
                default=None,
            )
            
            #print('the min syllable in floating',min_s,min_m)

            # Update the floating syllables with the last tone
            new_assoc = self.assoc[:]
            for i, (t, m, s) in enumerate(new_assoc):
                if s == min_s:
                    new_assoc[i] = (max_t, m, s)

            return new_assoc
        else:
            return None
    
    def float_syl_to_float_tone(self):
        
        """
        Associate the first floating syllable to the last tone.

        Example:
        ('HLH', [(1, 1, 1), (2, 1, 2), (2, 2, 2), (None, 1, 3), (None, 2, 3), (3, None, None)]) -->
        ('HLH', [(1, 1, 1), (2, 1, 2), (2, 2, 2), (3, 1, 3), (3, 2, 3)])
        """
        # Check if there are any floating syllables
        floating_syllables = [
            (t, m, s) for (t, m, s) in self.assoc if not t and s
        ]
        
        # Check if there are any floating syllables
        floating_tones = [
            (t, m, s) for (t, m, s) in self.assoc if t and not s
        ]
        new_assoc = self.assoc[:]

        if floating_syllables and floating_tones:
            min_s = min(s for t,m,s in floating_syllables)
            min_t = min(t for t,m,s in floating_tones)
            min_m = min(m for t,m,s in floating_syllables)
        
            
            # build a new association line
            
            for i in new_assoc:
                if i[0] == min_t:
                    new_assoc.remove(i)
                
                    
            for i, (t, m, s) in enumerate(new_assoc):
                if s == min_s and m == min_m:
                    new_assoc[i] = (min_t, m, s) 
                    
           
        
            return new_assoc        
        else:
            return None
        

    
    def show(self):
        print(self.ocp_mel,self.assoc) 

    def fully_spec(self):
        return all(t is not None and m is not None and s is not None for t, m, s in self.assoc)


    def __eq__(self, other):
        return self.ocp_mel == other.ocp_mel and set(self.assoc) == set(other.assoc)

    def add_assoc(self):
        """
        Add new associations by resolving floating tones or syllables.
        Returns a list of new Autorep objects with updated associations.
        """
        # Initialize new_ar as an empty list
        new_ar = []
 
        if self.float_syl_to_float_tone():
            new_ar.extend(Autorep(ocp_mel=self.ocp_mel, assoc=self.float_syl_to_float_tone()))

        if self.float_tone_to_syl():
            new_ar.extend(Autorep(ocp_mel=self.ocp_mel, assoc=self.float_tone_to_syl()))
        
        if self.float_syl_to_tone():
            print(self.float_syl_to_tone())
            new_ar.append(Autorep(ocp_mel=self.ocp_mel, assoc=self.float_syl_to_tone()))
        
        return new_ar

    
    def next_ar(self):
        next_ar = []

    # Handle the empty association case
        if not self.assoc:
            next_ar.extend([
                Autorep(ocp_mel="H", assoc=[(1, None, None)]),
                Autorep(ocp_mel="L", assoc=[(1, None, None)])
            ])
            for i in range(max_syl_weight):
                next_ar.append(Autorep(ocp_mel='', assoc=[(None, i + 1, 1)]))
        
        # Handle non-empty association case
        else:
        
            next_ar.extend(self.add_tone())  # Add tones if applicable

            for i in range(max_syl_weight):     
                next_ar.append(self.add_syl(i+1))
            
            if self.add_assoc():
                next_ar.extend(self.add_assoc())

        return next_ar
        
    
    def t_factor(self):
        tone_num = len(self.ocp_mel)
        return tone_num
    
    def s_factor(self):
        syl_num = max([k for _,_,k in self.assoc if k is not None], default=0)
        return syl_num     

    def k_factor(self):
        return self.t_factor() + self.s_factor()
    
    def __repr__(self):
        # This will be used when the object is inside a list
        return f"{self.ocp_mel}, {self.assoc}"
    

In [178]:
# Creating Autorep objects
a = Autorep("dú.hùu")  
b = Autorep("fà.dá.màa")  
c = Autorep("gáa.jì.màa.rée")  

# Testing containment
print("c contain a:",c.check_contain(a)) # False
print("c contain b:",c.check_contain(b)) # False
print("b contain a:",b.check_contain(a)) #   True
print("b contain c:",a.check_contain(c)) #  False
print("a contain b:",a.check_contain(b)) #  False
print("a contain c:",a.check_contain(c)) #  False

# Adding tone to b
d = b.add_tone()[0]
e = b.add_syl(2)
f = e.add_assoc()[0]
# Testing containment with modified b (d)
print("d contains b:", d.check_contain(b))  # Should print True
print("e contains b:", e.check_contain(b))  # Should print True
print("f contains e:", f.check_contain(e))  # Should print True


c contain a: False
c contain b: False
b contain a: True
b contain c: False
a contain b: False
a contain c: False
[(1, 1, 1), (2, 1, 2), (3, 1, 3), (3, 2, 3), (3, 1, 4), (3, 2, 4)]
d contains b: True
e contains b: True
f contains e: True


In [177]:
## regardless of association order, the function should return True
a = Autorep(ocp_mel="LH", assoc=[(1, None, None), (2, None, None)])
b = Autorep(ocp_mel="LH", assoc=[(2, 2, 1), (1, 1, 1)])
b.check_contain(a)  # Should print True



True

In [176]:
for i in b.next_ar():
    print(i.check_contain(a))

True
True
True
True


In [175]:
b.next_ar()

[LHL, [(1, 1, 1), (2, 2, 1), (3, None, None)],
 LH, [(1, 1, 1), (2, 2, 1), (None, 1, 2)],
 LH, [(1, 1, 1), (2, 2, 1), (None, 1, 2), (None, 2, 2)],
 LH, [(1, 1, 1), (2, 2, 1), (None, 1, 2), (None, 2, 2), (None, 3, 2)]]

In [118]:
from queue import Queue

def check_from_data(ar, positive_data):
    # Iterate over each autorep in the positive data
    for i in positive_data:
        # Check if the given autorep is a sub_structure of the current autorep in the loop
        if i.check_contain(ar):
            # If it is, return True immediately
            return True
    # If the loop completes without finding any match, return False
    return False        


def check_from_grammar(ar, grammar):
    # Iterate over each autorep in the positive data
   for i in grammar:
      # Check if the given autorep contains the structure in the grammar
      if ar.check_contain(i):
         # If it is, then the grammar check fails
         return False
   # If the loop completes without finding any match, return True
   return True

 
def bufia(D, t = 2, s = 2):
   D = autolis
   t_threshold = t
   s_threshold = s

   Q = Queue()
   s0 = Autorep()  
   V = []
   G = []
   Q.put(s0)

   while not Q.empty():
      s = Q.get()
      V.append(s)
      if check_from_data(s, D):
         S = s.next_ar()
         for i in S:
            if i not in V and check_from_grammar(i, G) and i.t_factor() <= t_threshold and i.s_factor() <= s_threshold:
               Q.put(i)
      else:
         if check_from_grammar(s, G):
               # only append to the grammar if it is not a super_structure of any autorep in the grammar
               G.append(s)


   print(f'found {len(G)} constraints')
   return G 